# 밑바닥부터 시작하는 딥러닝 2


챕터1
---



1.1 수학과 파이썬 복습
---



벡터는 1차원 배열, 행렬은 2차원 배열

벡터는 숫자가 일렬로 늘어선 집합으로 표현가능. 크기와 방향을 가진 양.
행렬은 숫자가 2차원 형태로 늘어선 것

벡터를 표현하는 방법  : 세로로 열 벡터, 가로로 행 백터

수학과 딥러닝등 많은 분야에서 열벡터 방식을 선호하지만, 책에선 일단 구현 편의를 위해 행벡터를 다루겠음

벡터나 행렬은 굵게 표시하기. (스칼라 와 구분)

In [1]:
import numpy as np

x = np.array([1 ,2, 3])

print(x.__class__) # 클래스 이름 표시
print(x.shape)
print(x.ndim)

<class 'numpy.ndarray'>
(3,)
1


In [2]:
W = np.array([[1, 2, 3], [4, 5, 6]])

print(W.shape)
print(W.ndim)

(2, 3)
2


행렬의 element wise operation

In [3]:
W = np.array([[1, 2, 3], [4, 5, 6]])
X = np.array([[0, 1, 2], [3, 4, 5]])

print(W+X)
print(W*X)

[[ 1  3  5]
 [ 7  9 11]]
[[ 0  2  6]
 [12 20 30]]


브로드캐스트

In [4]:
A = np.array([[1, 2], [3, 4]])
A * 10
# 스칼라 값이 2*2 행렬로 확장된 후 원소별 연산 수행

array([[10, 20],
       [30, 40]])

In [5]:
A = np.array([[1, 2], [3, 4]])
B = np.array([10, 20])
A * B
# 1차월 배열 B 가 값이 2 차원 배열 A와 shape 가 같도록 확장된 후 원소별 연산 수행

array([[10, 40],
       [30, 80]])

벡터의 내적과 행렬의 곱

내적은 각 원소를 대응하여 곱한 후 더한 것 - 두 벡터가 얼마나 같은 방향을 향하고 있는지. 같은 방향의 내적은 +1, 반대일시 -1

In [6]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(np.dot(a, b))

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
print(np.matmul(A, B))


32
[[19 22]
 [43 50]]



신경망의 추론
---



fully connect layer 의 미니 배치 학습 - checking shape of each matrices

In [0]:
import numpy as np

W1 = np.random.randn(2, 4) # 가중치
b1 = np.random.randn(4) # 편향
x = np.random.randn(10, 2) # 입력
h = np.matmul(x, W1) + b1 


In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x = np.random.randn(10, 2)
W1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
W2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(a, W2) + b2
print(s.shape)

(10, 3)


Fully connected layer 에 의한 변환을 Affine 계층이라고 부름 - 기하학의 Affine 변환에 해당

***구현규칙***


forward(), backward() 메서드 구현, params 는 매개변수 담는 리스트, grads 는 params 의 매개변수에 대응하여, 해당 매개변수의 기울기를 보관하는 리스트

* 이렇게 구현 규칙을 세워두고 구현하면 코드가 일관되고 확장성이 좋아짐. 아래 코드는 구현규칙을 따라서 모든 계층에는 학습해야하는 매개변수가 반드시 인스턴스 변수인 params 에 존재하게 됨. 이 덕분에 신경망의 모든 매개변수를 간단하게 정리 할 수 있고, 자연스럽게 매개변수 갱신 작업이나 매개변수를 파일로 저장하는 일이 쉬워짐

In [0]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

In [0]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치와 편향 초기화
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)
        
        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        # 모든 가중치를 리스트에 모은다.
        self.params = []
        for layer in self.layers:
            self.params += layer.params # 나오는 매개변수들을 하나의 리스트에 보관하기 위해 계속 더해줌. 더하면 결합
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x    
            
        

In [12]:
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)
print("Shape of matrix after inference : {}".format(s.shape))

Shape of matrix after inference : (10, 3)



1.3 신경망의 학습
---



수학에서는 기울기를 벡터에 대한 미분으로 한정하지만, 딥러닝에서는 행렬과 텐서에 대해서도 미분을 정의하고 그것을 기울기라 부름

Computational Graph
- 덧셈노드 : 상류에서온 기울기를 그대로 보냄. 
- 곱셈노드 : 상류에서온 기울기를 서로 입력의 반대값을 곱함
- 분기노드 : 상류에서온 두 기울기를 더함

* Repeat 노드 : N 개로 분기하는 노드 

길이가 D 인 배열을 D * N 의 형태로 분기시킴.

In [13]:
import numpy as np
D, N = 8, 7

x = np.random.randn(1, 0) # 입력
y = np.repeat(x, N, axis=0) # 순전파

dy = np.random.randn(N, D) # 무작위 기울기
dx = np.sum(dy, axis=0, keepdims=True) # 역전파

print(dy.shape)
print(dx.shape) # keepdims 가 True 일시 shape(1, N), False 일시 1차원으로 바뀌므로 (N, )


(7, 8)
(1, 8)


* Sum 노드

N * D 배열 에 대해 총합을 0축(행)에 대해 구하는 계산하기 - 즉 결과는 1 * D 의 형상으로 나올 것임


In [14]:
import numpy as np
D, N = 8, 7

x = np.random.randn(1, 0) # 입력
y =  np.sum(x, axis=0, keepdims=True) # 순전파

dy = np.random.randn(N, D) # 무작위 기울기
dx = np.repeat(dy, N, axis=0) # 역전파

print(dy.shape)
print(dx.shape) # keepdims 가 True 일시 shape(1, N), False 일시 1차원으로 바뀌므로 (N, )


(7, 8)
(49, 8)


* MatMul 노드

y = x W  - x( 1* D), W(D * H), y(1 * H)  형상

![image](https://user-images.githubusercontent.com/46298038/61572977-bc98ba00-aae1-11e9-9311-d76ff260435d.png)


dL/dx   = dL/dy * W^t\
(N * D) = (N * H) (H * D)


dL/dW   = X^t * dL/dy\
(N * D) = (D * N) (N * H)


In [0]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW # 그냥 할당하지 않고(얕은 복사) 덮어써서(깊은 복사 수행) 
        return dx

인스턴스 변수 grads 를 위와 같잉 [...] 생략기호를 통해 변수의 메모리 주소를 고정함으로써 다루기 더 쉬워짐.

추가 설명 : grads 리스트는 각 매개변수의 기울기를 저장하는데, 리스트의 각 원소는 넘파이 배열이며, 계층을 생성할 때 한 번만 생성됨. 그 이후로는 항상 '생략 기호'를 이용하기 때문에 이 넘파이 배열의 메모리 주소가 변하는 일 없이 항상 값을 덮어씀. 이렇게 하면 기울기를 그룹화 하는 작업을 최초에 한번만 하면 된다는 이점이 생김

![image](https://user-images.githubusercontent.com/46298038/61573926-9b3fca00-aaf1-11e9-89ca-0496785a6001.png)


In [0]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
       

![image](https://user-images.githubusercontent.com/46298038/61573930-ab57a980-aaf1-11e9-8f02-4c36cae3cdf1.png)


In [0]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
        return out
        
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis = 0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx    
    

Softmax with Loss

![image](https://user-images.githubusercontent.com/46298038/61573953-30db5980-aaf2-11e9-9659-0691e726612a.png)

In [0]:
def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 정답 데이터가 원핫 벡터일 경우 정답 레이블 인덱스로 변환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [0]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None # Softmax \의 출력
        self.t = None # 정답 레이블
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        
        # 정답 레이블이 원핫 벡터일 경우 정답의 인덱스로 변환
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
            
        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        
        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size
        
        return dx
    

Update weights

In [0]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]
        

수행될 psuedo code

In [0]:
# model = TwoLayerNet(...)
# optimizer = SGD()

# for i in range(1000):
#     '''
#     x_batch, t_batch = get_mini_batch() # 미니 배치 획득
#     loss = model.forward(x_batch, t_batch)
#     model.backward()
#     optimizer.update(model.params, model.grads)
    
#     '''



1.4 실전 문제 해결 - 스파이럴 데이터셋
---



spiral dataset 

In [0]:
import numpy as np


def load_data(seed=1984):
    np.random.seed(seed)
    N = 300  # 클래스당 샘플 수
    DIM = 2  # 데어터 요소 수
    CLS_NUM = 3  # 클래스 수

    x = np.zeros((N*CLS_NUM, DIM))
    t = np.zeros((N*CLS_NUM, CLS_NUM), dtype=np.int)

    for j in range(CLS_NUM):
        for i in range(N): # N*j, N*(j+1)):
            rate = i / N
            radius = 1.0*rate
            theta = j*4.0 + 4.0*rate + np.random.randn()*0.2

            ix = N*j + i
            x[ix] = np.array([radius*np.sin(theta),
                              radius*np.cos(theta)]).flatten()
            t[ix, j] = 1

    return x, t

In [43]:
# import sys
# sys.path.append() # 부모 디렉토리의 파일을 가져올 수 있도록 설정

# from dataset import spiral
import matplotlib.pyplot as plt

# x, t = spiral.load_data()
x, t = load_data()

print(x.shape)
print(t.shape)


(900, 2)
(900, 3)


신경망 구현

In [0]:
# import sys
# sys.path.append('..')
# import numpy as np
# from common.layers import Affine, Sigmoid, SoftmaxWithLoss

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치와 편향 초기화
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)
        
        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        self.loss_layer = SoftmaxWithLoss()
        
        # 모든 가중치와 기울기를 리스트에 모은다
        
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout = 1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
        


학습용 코드
---



In [47]:
# import sys
# sys.path.appedn('..')
# import numpy as np
# from common.optimizer import SGD
# from dataset import spiral
# import matplotlib.pyplot as plt
# from two_layer_net import TwoLayerNet

# 하이퍼 파라메터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
learning_rate = 1.0


# 데이터 읽기, 모델과 옵티마이저 생성
x, t = load_data() # 원래는 spiral.load_data()
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

# 학습에 사용하는 변수
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # 데이터 뒤섞기
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]
    
    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]
        
        # 기울기를 구해서 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

        # 정기적으로 학습 결과 출력

        if (iters + 1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print('에폭 : %d, 반복 : %d / %d, 에폭 : %.2f '% (epoch + 1, iters + 1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0
    


에폭 : 1, 반복 : 10 / 30, 에폭 : 1.14 
에폭 : 1, 반복 : 20 / 30, 에폭 : 1.14 
에폭 : 1, 반복 : 30 / 30, 에폭 : 1.14 
에폭 : 2, 반복 : 10 / 30, 에폭 : 1.14 
에폭 : 2, 반복 : 20 / 30, 에폭 : 1.12 
에폭 : 2, 반복 : 30 / 30, 에폭 : 1.15 
에폭 : 3, 반복 : 10 / 30, 에폭 : 1.15 
에폭 : 3, 반복 : 20 / 30, 에폭 : 1.14 
에폭 : 3, 반복 : 30 / 30, 에폭 : 1.19 
에폭 : 4, 반복 : 10 / 30, 에폭 : 1.13 
에폭 : 4, 반복 : 20 / 30, 에폭 : 1.11 
에폭 : 4, 반복 : 30 / 30, 에폭 : 1.09 
에폭 : 5, 반복 : 10 / 30, 에폭 : 1.08 
에폭 : 5, 반복 : 20 / 30, 에폭 : 1.02 
에폭 : 5, 반복 : 30 / 30, 에폭 : 1.01 
에폭 : 6, 반복 : 10 / 30, 에폭 : 0.96 
에폭 : 6, 반복 : 20 / 30, 에폭 : 0.92 
에폭 : 6, 반복 : 30 / 30, 에폭 : 0.90 
에폭 : 7, 반복 : 10 / 30, 에폭 : 0.84 
에폭 : 7, 반복 : 20 / 30, 에폭 : 0.82 
에폭 : 7, 반복 : 30 / 30, 에폭 : 0.79 
에폭 : 8, 반복 : 10 / 30, 에폭 : 0.77 
에폭 : 8, 반복 : 20 / 30, 에폭 : 0.78 
에폭 : 8, 반복 : 30 / 30, 에폭 : 0.78 
에폭 : 9, 반복 : 10 / 30, 에폭 : 0.76 
에폭 : 9, 반복 : 20 / 30, 에폭 : 0.74 
에폭 : 9, 반복 : 30 / 30, 에폭 : 0.74 
에폭 : 10, 반복 : 10 / 30, 에폭 : 0.77 
에폭 : 10, 반복 : 20 / 30, 에폭 : 0.75 
에폭 : 10, 반복 : 30 / 30, 에폭 : 0.70 
에폭 : 11

Trainer 클래스 // 신경망 계산 고속화(비트정밀도, GPU ) 안다룸